In [1]:
import pandas as pd
import numpy as np 

In [2]:
# Load the data into dataframes
file_names = [
    "TB Data 2000-2022 Race-Ethnicity",
    "TB Data 2000-2022 M-F",
    "TB Data 2000-2022 US vs Non",
    "TB Data 2000-2022 Age Groups"
]

dataframes = {}

for file_name in file_names:
    # Load data from CSV file
    df = pd.read_csv(f"resources/{file_name}.csv")
    
    # Add "_cases" suffix to dataframe name
    df_name = f"{file_name}_cases"
    
    # Rename columns if needed
    df.columns = df.columns.str.strip()  # Remove leading/trailing whitespaces
    
    # Add dataframe to the dictionary
    dataframes[df_name] = df


In [3]:
# Display .head() of each dataframe
for df_name, df in dataframes.items():
    print(f"DataFrame: {df_name}")
    print(df.head())
    print("\n")


DataFrame: TB Data 2000-2022 Race-Ethnicity_cases
      Indicator  Year Geography  FIPS                 Race/Ethnicity Cases  \
0  Tuberculosis  2000   Alabama     1  American Indian/Alaska Native     1   
1  Tuberculosis  2000   Alabama     1                          Asian     7   
2  Tuberculosis  2000   Alabama     1         Black/African American   177   
3  Tuberculosis  2000   Alabama     1                Hispanic/Latino    12   
4  Tuberculosis  2000   Alabama     1                    Multiracial     0   

  Rate per 100000  
0             4.6  
1              22  
2            15.3  
3            15.7  
4               0  


DataFrame: TB Data 2000-2022 M-F_cases
      Indicator  Year Geography  FIPS     Sex Cases  Rate per 100000
0  Tuberculosis  2000   Alabama     1  Female    91              4.0
1  Tuberculosis  2000   Alabama     1    Male   219             10.2
2  Tuberculosis  2001   Alabama     1  Female    81              3.5
3  Tuberculosis  2001   Alabama     1    Mal

In [4]:
# Duplicate dataframes and rename them with "_rate" suffix
for df_name, df in dataframes.copy().items():
    df_copy = df.copy()
    df_copy.columns = [col.replace("_cases", "_rate") for col in df_copy.columns]
    dataframes[df_name.replace("_cases", "_rate")] = df_copy


In [5]:
# Drop "Rate per 100000" column from "_cases" dataframes
for df_name in dataframes.copy():
    if "_cases" in df_name:
        dataframes[df_name] = dataframes[df_name].drop(columns=["Rate per 100000"])

# Drop "Cases" column from "_rate" dataframes
for df_name in dataframes.copy():
    if "_rate" in df_name:
        dataframes[df_name] = dataframes[df_name].drop(columns=["Cases"])


In [6]:
# Display .head() of each modified dataframe
for df_name, df in dataframes.items():
    print(f"DataFrame: {df_name}")
    print(df.head())
    print("\n")


DataFrame: TB Data 2000-2022 Race-Ethnicity_cases
      Indicator  Year Geography  FIPS                 Race/Ethnicity Cases
0  Tuberculosis  2000   Alabama     1  American Indian/Alaska Native     1
1  Tuberculosis  2000   Alabama     1                          Asian     7
2  Tuberculosis  2000   Alabama     1         Black/African American   177
3  Tuberculosis  2000   Alabama     1                Hispanic/Latino    12
4  Tuberculosis  2000   Alabama     1                    Multiracial     0


DataFrame: TB Data 2000-2022 M-F_cases
      Indicator  Year Geography  FIPS     Sex Cases
0  Tuberculosis  2000   Alabama     1  Female    91
1  Tuberculosis  2000   Alabama     1    Male   219
2  Tuberculosis  2001   Alabama     1  Female    81
3  Tuberculosis  2001   Alabama     1    Male   183
4  Tuberculosis  2002   Alabama     1  Female    85


DataFrame: TB Data 2000-2022 US vs Non_cases
      Indicator  Year Geography  FIPS Country of birth Cases
0  Tuberculosis  2000   Alabama     1  

In [7]:
def clean_dataframes(dataframes):
    for df_name, df in dataframes.copy().items():
        if "M-F_cases" in df_name:
            dataframes[df_name] = process_cases(df, ['Year', 'Geography', 'Sex'])

        elif "M-F_rate" in df_name or "US vs Non_rate" in df_name:
            dataframes[df_name] = process_rate(df, ['Year', 'Geography', 'Sex'] if "M-F_rate" in df_name else ['Year', 'Geography', 'Country of birth'])

        elif "US vs Non_cases" in df_name:
            dataframes[df_name] = process_cases(df, ['Year', 'Geography', 'Country of birth'])

        elif "Age Groups_cases" in df_name:
            dataframes[df_name] = process_cases(df, ['Year', 'Geography', 'Age Group'])

        elif "Age Groups_rate" in df_name:
            dataframes[df_name] = process_rate(df, ['Year', 'Geography', 'Age Group'])

        elif "Race-Ethnicity_cases" in df_name:
            dataframes[df_name] = process_cases(df, ['Year', 'Geography', 'Race/Ethnicity'])

        elif "Race-Ethnicity_rate" in df_name:
            dataframes[df_name] = process_rate(df, ['Year', 'Geography', 'Race/Ethnicity'])

def process_cases(df, groupby_cols):
    df_cases_split = df.groupby(groupby_cols)['Cases'].sum().unstack() #allows the unique values to be indexed as columns
    df_cases_split.reset_index(inplace=True)
    df_cases_split.drop_duplicates(subset=["Year"], inplace=True)
    return df_cases_split

def process_rate(df, groupby_cols):
    df_rate_split = df.groupby(groupby_cols)['Rate per 100000'].sum().unstack()
    df_rate_split.reset_index(inplace=True)
    df_rate_split.drop_duplicates(subset=["Year"], inplace=True)
    return df_rate_split

# Call the function to clean the dataframes
clean_dataframes(dataframes)

In [8]:
dataframes["TB Data 2000-2022 Age Groups_rate"].rename(columns={"14-May": "5-14"}, inplace=True)
dataframes["TB Data 2000-2022 Age Groups_cases"].rename(columns={"14-May": "5-14"}, inplace=True)

In [9]:
# Merge the _cases dataframes
cases_dfs = [df for df_name, df in dataframes.items() if "_cases" in df_name]
merged_cases_df = pd.concat(cases_dfs, ignore_index=True)
merged_cases_df = merged_cases_df.groupby(['Year', 'Geography']).sum().reset_index()

# Sort the dataframe by Year and Geography
merged_cases_df = merged_cases_df.sort_values(by=['Year', 'Geography'])

# Display the head of the merged _cases dataframe
print("Merged _cases dataframe:")
print(merged_cases_df.head())
print("\n")


Merged _cases dataframe:
   Year Geography American Indian/Alaska Native Asian Black/African American  \
0  2000   Alabama                             1     7                    177   
1  2001   Alabama                             0    10                    127   
2  2002   Alabama                             1    11                    118   
3  2003   Alabama                             0    10                    143   
4  2004   Alabama                             0    17                    117   

  Hispanic/Latino Multiracial Native Hawaiian/Other Pacific Islander Unknown  \
0              12           0                                      0       0   
1              21           0                                      0       0   
2              16           0                                      0       0   
3              20           0                                      0       0   
4              18           0                                      0       0   

  White  ... 

In [10]:
# Merge the _rate dataframes
rate_dfs = [df for df_name, df in dataframes.items() if "_rate" in df_name]
merged_rate_df = pd.concat(rate_dfs, ignore_index=True)
merged_rate_df = merged_rate_df.groupby(['Year', 'Geography']).sum().reset_index()


# Sort the dataframe by Year and Geography
merged_rate_df = merged_rate_df.sort_values(by=['Year', 'Geography'])

# Display the head of the merged _rate dataframe
print("Merged _rate dataframe:")
print(merged_rate_df.head())
print("\n")



Merged _rate dataframe:
   Year Geography American Indian/Alaska Native Asian Black/African American  \
0  2000   Alabama                           4.6    22                   15.3   
1  2001   Alabama                             0  30.4                     11   
2  2002   Alabama                           4.6  32.1                   10.1   
3  2003   Alabama                             0  27.8                   12.2   
4  2004   Alabama                             0  44.6                     10   

  Hispanic/Latino Multiracial Native Hawaiian/Other Pacific Islander  \
0            15.7           0                                      0   
1            25.3           0                                      0   
2            17.9           0                                      0   
3            20.8           0                                      0   
4            17.4           0                                      0   

              Unknown White  ...       Non-U.S.–born          

In [11]:
merged_cases_df.to_csv("resources/merged_cases.csv", index=False)
merged_rate_df.to_csv("resources/merged_rate.csv", index=False)